# Electricity Price Forecasting Using an LSTM-RNN
(Data dowloaded from https://www.kaggle.com/salilchoubey/electrity-prices on December 16th 2021)

*Made-up Case*: A data center in Ireland consumes large amounts of electricity to keep their operation running. So much, in fact, that buying electrical power makes up a significant part of their total spending. Recently, sections of the data center have been upgraded to be put in and out of a standby state. The sections run on significantly reduced power in this state, and they can be switched in and out in only a few minutes. The purpose of this recent upgrade is to reduce overall spending by turning of non-essential equipment in times of peak electricity prices.

The price of electricity in the area is determined by many different factors but, generally speaking, it will spike when the national demand increases. In afternoon hours, people come home from work, turn on their lights, tellys, tea kettles, and other common household appliances. This causes a large increase demand for power, and consequently, a spike in price. The spike can be several hundreds of percent above the baseline price level. Reducing consumption in these peak hours would lead to reduced spending.

The price spikes do not appear every day, and sometimes it is not possible to use the standby state due to client demands. To employ the standby-state succesfully, it is therefore nescessary to forecast the electricity price with sufficient accuracy. The data center has recorded the actual price alongside several other parameters in half-hour intervals during the last two years. 

The task at hand is to use the data to train an appropriate Machine Learning Model that can predict the price more accurately than the already available national forecast. Here, we choose to use a Long-Short Term Memory (LSTM) based Recurrent Neural Network (RNN).

In [ ]:
#Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential
from tensorflow.keras import layers

#Flags
holiday_removed = 0
year_joined = 0
scaled = 0

#Labels
days_name = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
#Path to data set
path = r'C:\ElectricityPriceForecasting/'
dataset = 'electricity_prices.csv'

#Data loaded into pandas dataframe. Missing entries denoted by ? in csv.file
data = pd.read_csv(path + dataset, na_values='?')

#Overview of dataframe
print('#samples  (rows)    =  {}'.format(data.shape[0]))
print('#features (columns) =  {}'.format(data.shape[1]))
data.head(5)

### Features
As seen above, the data set is comprised of 38014 samples (rows) and 18 features (columns). The first 9 features are identity or time features for a given sample. The following 8 features are metadata and the last is the price to be forecasted. A detailed description of each features is given below:

#### Time features
- DateTime: (String) Defines date and time of samples
- Holiday: (String) States the name of the holiday if the date is a bank holiday
- HolidayFlag: (Integer) Flag for bank holiday; 1 if holiday, 0 if not
- DayOfWeek: (Integer) Identifier for day of the week; 0 to 6 for Monday through Sunday
- WeekOfYear: (Integer) Identifier for week of the year; 0 to 52
- Day: (Integer) Day of the date in DateTime
- Month: (Integer) Month of the date in DateTime
- Year: (Integer) Year of the date in DateTime
- PeriodOfDay: (Interger) Identifier for the half-hour interval of a given day; 0 to 47

#### Metadata features
- ForecastWindProduction: (Float) Forecasted power production from wind
- SystemLoadEA: (Float) Forecast of the power grid system load
- SMPEA: (Float) National forecast of the electricity price
- ORKTemperature: (Float) Actual temperature measured at Cork Airport
- ORKWindspeed: (Float) Actual wind speed measured at Cork Airport
- CO2Intensity: (Float) Actual CO2 intensity of power production in g/kWh
- ActualWindProduction: Actual power production from wind
- SystemLoadEP2: (Float) Actual power grid system load
- SMPEP2: (Float) The actual price of electricity (price to be forecasted)


In [ ]:
time_features = [
    'PeriodOfDay',
    'DayOfWeek',
    'WeekOfYear',
    'Day',
    'Month',
    'Year'   
]

metadata_features = [
    'SMPEP2',
    'ForecastWindProduction',
    'SystemLoadEA',
    'SMPEA',
    'ORKTemperature',
    'ORKWindspeed',
    'CO2Intensity',
    'ActualWindProduction',
    'SystemLoadEP2'
]

### Price Forecasting
A forecast of the price of electricity is already given by a national institution (the SMPEA feature). The goal of this project is thus to implement a Machine Learning Model that is more accurate than the forecast. Below, the forecasted and actual price of three random days are plotted. The two days to the left both have an afternoon spike in price but the one to the right does not.

The root mean square error (RMSE) of the forecasted price is 29.77, the mean absolute error (MAE) is 14.95, and the R-squared (R2) value is 0.149. Compare these errors to the mean target price for the entire time period, which is 64.14. The MAE of the forecasted price is thus nearly a quarter of the mean target price, suggesting a relatively inaccurate forecast.

In [ ]:
#Typecast DateTime column into pandas.timestamp
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d/%m/%Y %H:%M')
dates = data['DateTime'].apply(lambda x: x.date()).unique()

#Plot forecasted and actual price from three random days (default seed = 42)
fig, axes = plt.subplots(1,3, figsize = (12,5))
fig.subplots_adjust(wspace = 0.25)

rs = np.random.seed(42)
for ax in axes:
    rand_date = dates[np.random.randint(0,len(dates))]
    price     = data['SMPEP2'][data['DateTime'].apply(lambda x: x.date()) == rand_date]
    foreprice = data['SMPEA'][data['DateTime'].apply(lambda x: x.date()) == rand_date]
    
    sns.lineplot(x = np.array(range(0,len(price)))/2, y=price,     ax=ax, label='Actual price')
    sns.lineplot(x = np.array(range(0,len(price)))/2, y=foreprice, ax=ax, label='Forecasted price')
        
    ax.set_xlabel('Hour of day'); ax.set_ylabel('SMPEP2')
    ax.set(xlim=(0,23), ylim=(0, max(max(price), max(foreprice))*1.10))
    ax.legend()
    ax.title.set_text(days_name[rand_date.weekday()] + ' ' + str(rand_date))
    

#Compute and report evaluation metrics for forecasted price
mean_target_price = data['SMPEP2'].mean()

RMSE_fore = np.sqrt(1/len(data['SMPEP2'])*np.sum(np.power(data['SMPEP2'] - data['SMPEA'],2)))
MAE_fore = 1/len(data['SMPEP2'])*np.sum(abs(data['SMPEP2'] - data['SMPEA']))
R2_fore = 1 - np.sum(np.power(data['SMPEA'] - data['SMPEP2'], 2))/np.sum(np.power(data['SMPEA'] - mean_target_price, 2))

print('FORECASTED VS. ACTUAL PRICE')
print('Mean target price:      {:.2f}'.format(mean_target_price))
print('Root mean square error: {:.2f}'.format(RMSE_fore))
print('Mean absolute error:    {:.2f}'.format(MAE_fore))
print('R-squared:              {:.3f}'.format(R2_fore))

### Data Preprocessing
Before the data can be fed to a Machine Learning Model, it has to be properly cleaned and encoded. Several samples have missing entries and, in some cases, no metadata features have been recorded at all. For instance, entry no. 24769 recorded on March 31st 2013 at 01.30 has all missing metadata features. This sample was recorded on Easter in 2013. Examination of the data set reveals that holidays are often problematic with relatively many missing entries.

In [ ]:
#Entry no. 24769
print(data.loc[24769,:])

In [ ]:
#Function to find missing values. Returns dataframe
def missing_values(df):
    feature = []
    dtype = []
    missing_values = []
    unique_values = []
    holi_missing_rate = []
    
    for col in df.columns:
        feature.append(col)
        dtype.append(df[col].dtype)
        missing_values.append(df[col].isnull().sum())
        unique_values.append(len(df[col].unique()))
        
        if data[col].isnull().sum() > 0:
            holi_missing_rate.append(round(holiday_missing_rate(df, col),2))
        else:
            holi_missing_rate.append(0)
        
    dataframe_missing_info = pd.DataFrame({
        'feature': feature,
        'dtype': dtype,
        '#missing values': missing_values,
        'unique values': unique_values,
        'holiday missing rate (%)': holi_missing_rate
    })
        
    return dataframe_missing_info


#Function to find percentage of missing entries on holidays. Returns float.
def holiday_missing_rate(dataframe, col):
    flags = dataframe['HolidayFlag'][dataframe[col].isnull()]
    nonholi = len(flags[flags == 0])
    holi = len(flags[flags == 1])
    holiday_missing_rate = holi/(nonholi + holi)*100
    
    return holiday_missing_rate

#Utilization of above functions to compute missing values dataframe
data_missing = missing_values(data)
data_missing

#### Removal of holidays
As seen in the right-most column above, many of the missing entries fall on holidays. There are 32 holidays out of a total of 792 days (see below), which correspond to ~4% of the data set. However, for the missing entries in ORKTemperature, 39.32% fall on holidays. 

We therefore decide to remove samples that fall on holidays. This will exclude many of the missing entries and also reduce the dimensionality of the dataset. Also, consider the fact that the Easter holidays, for instance, only have been sampled twice in the data set. 

In [ ]:
#Find and report number of holidays
if holiday_removed == 0:
    days = round(data.shape[0]/48)
    holidays = round(len(data[data['HolidayFlag'] == 1].index)/48)
    holiday_removed += 1
    
    #Remove holidays from dataframe
    holiday_idxs = data[data['HolidayFlag'] == 1].index
    data = data.drop(holiday_idxs)
    
print('Total days: {}'.format(days))
print('Total holidays: {}'.format(holidays))
print('Holiday rate: {:.3}%'.format(holidays/days*100))


print('\nTotal normal days: {}'.format(round(data.shape[0]/48)))

#Compute missing values dataframe after holiday removal
data_missing = missing_values(data)
data_missing

#### Imputation of missing entries
After removal of the holidays, there are still many missing entries, especially in the ORKTemeprature and ORKWindspeed feature. The missing values are imputed by linear interpolation, meaning that one or a series of missing values will be determined by a linear interpolation between the bounding values of the missing value(s).

In [ ]:
#Impute missing data. Missing entries will be filled by linear interpolation.
for col in data_missing[data_missing['#missing values'] > 0]['feature']:
    data[col].interpolate(method='linear', inplace=True)

data_missing = missing_values(data)
data_missing

#### Sampling distribution
Now that the data have been cleaned, it is time to get an overview of the distribution of samples across the different time features. The histograms below show that counts for PeriodOfDay (top left) and day-of-month (bottom left) are equally distributed across the bins. The sample count on Mondays (DayOfWeek = 0, top middle) is slightly less due to the exclusion of holidays, which happen to fall inproportionally on Mondays in the time period. Only the end of 2011 has been sampled (bottom right). This leads to more counts for weeks (top right) and months (bottom middle) at the end of the year.

In [ ]:
#Plot histograms for time features
fig, axes = plt.subplots(2, 3, figsize=(12,10))
fig.subplots_adjust(wspace=0.35)
axes = axes.ravel()

for i, feature in enumerate(time_features):
    bins = int(data_missing['unique values'][data_missing['feature'] == feature])
    sns.histplot(data[feature], bins=bins, ax=axes[i])

#### Encoding of time features
To properly train a Machine Learning Model for time series forecasting, it is important to handle the time features adequately. The task is to filter out non-significant or redundant time features, and encode significant ones in a meaningful way that the Machine Learning Model can interpret.

#### Overview of price disitrbution
We begin by visualizing the distribution of prices along the different time features. As shown in the figures below, the target price (SMPEP2) depends significantly on the PeriodOfDay with low prices at night and a spike in the afternoon hours (top left). The price also depends on the WeekOfYear with higher price spikes in the Winter (top right). Furthermore, there seems to be a trend of increasing prices over the three years (bottom right). At first glance, there does not seem to be a clear trend for DayOfWeek or Day of the month.

In the following sections, we shall examine the price distributions accross the different time features in more detail.

In [ ]:
#Plot scatter plots for the target price (SMPEP2) versus the different time features
fig, axes = plt.subplots(2, 3, figsize=(12, 10))
fig.subplots_adjust(wspace=0.35)
axes = axes.ravel()

for i, feature in enumerate(time_features):
    sns.scatterplot(data=data, x=feature, y='SMPEP2', ax=axes[i])

# There is an outlier in the price column with an exact value of 1000.00. 
# This is manually changed to the mean between bounding values
outlier_index = data['SMPEP2'][data['SMPEP2'] > 900].index

if outlier_index.shape[0] > 0:
    outlier_index = outlier_index.values[0]
    new_val = 0.5*(data.at[outlier_index - 1, 'SMPEP2'] + data.at[outlier_index + 1, 'SMPEP2'])
    data.at[outlier_index, 'SMPEP2'] = new_val

#### DayOfWeek feature
As seen below, the mean prices on a given day of the week does actually variy from 69.13 on Mondays to 60.10 on Saturdays. In general, the price on Mondays is significantly higher than on other weekdays. Also, the spike in prices tend to be higher on Tuesdays through Thurdays, but lower in the weekend.

These observations lead to one-hot encoding of 3 categories (instead of 7) for the DayOfWeek feature:
- Mondays (monday)
- Tuesday, Wednesday, Thursday (midweek)
- Friday, Saturday, Sunday (weekend)

In [ ]:
#Compute mean prices for every day of the week
mean_price = []
for day in list(range(0,7)):
    mean_price.append(data['SMPEP2'][data['DayOfWeek'] == day].mean())

#Seaborn boxen plot to visualize the price distribution for each day of the week
fig, axes = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)
fig.subplots_adjust(hspace=0)
axes[0].plot(list(range(0, 7)), mean_price, 'k.--')
sns.boxenplot(data=data, x='DayOfWeek', y='SMPEP2', ax=axes[1])
axes[0].set(ylim=(59, 71), yticks=[61, 63, 65, 67, 69], ylabel='Mean')
axes[0].set_title('Price distribution across weekdays');
axes[1].set(ylim=(-25, 650));
axes[1].set_xticklabels(days_name)

#One-hot encode to three categories for DayOfWeek
day_one_hot = pd.get_dummies(data['DayOfWeek'])

data['monday'] = day_one_hot[0]
data['midweek'] = day_one_hot.iloc[:, 1:4].sum(axis=1)
data['weekend'] = day_one_hot.iloc[:, 4:7].sum(axis=1)

data.head(1)

#### Day (of month) feature
The mean prices on a given day of a month fluctuate between 60 and 68 but do not seem to have any significant trend. For reasons unclear, price spikes seem to be espcially low on the 3rd, 17th, and 31th day of the month. Overall, these observeations lead to the exclusion of the Day of month feature from the modelling.

In [ ]:
#Compute mean prices for every day of the month
mean_price = []
for day in list(range(1,32)):
    mean_price.append(data['SMPEP2'][(data['Day'] == day) & (data['Year'] != 2011)].mean())

#Seaborn boxen plot to visualize the price distribution for each day of the month
fig, axes = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)
fig.subplots_adjust(hspace=0)
axes[0].plot(list(range(0, 31)), mean_price, 'k.--')
sns.boxenplot(data=data[data['Year'] != 2011], x='Day', y='SMPEP2', ax=axes[1])
axes[0].set(ylim=(60, 68), yticks = [61, 63, 65, 67], ylabel='Mean')
axes[1].set(ylim=(-25, 650));
axes[0].set_title('Price distribution across days of the month');

#### Month and WeekOfYear features
The price spikes seem to be higher in the Winter months (October to April) than in the Summer. Therefore, the month column is one-hot encoded into two categories: winter (Oct, Nov, Dec, Jan, Feb, Mar) and summer (Apr, May, Jun, Jul, Aug,  Sep). These two categories are assumed to also encompass the similar trend observed in the WeekOfYear feature.

In [ ]:
#Compute mean prices for each month
mean_price = []
for month in list(range(1, 13)):
    mean_price.append(data['SMPEP2'][(data['Month'] == month) & (data['Year'] != 2011)].mean())

fig, axes = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)
fig.subplots_adjust(hspace=0)
axes[0].plot(np.arange(0, 12), mean_price, 'k.--')
sns.boxenplot(data=data[data['Year'] != 2011], x='Month', y='SMPEP2', ax=axes[1])
axes[0].set(ylim=(55, 75), yticks=[57.5, 62.5, 67.5, 72.5], ylabel='Average')
axes[1].set(ylim=(-25, 650));
axes[0].set_title('Price distribution across months');


#One-hot encoding of the two categories of Month
month_one_hot = pd.get_dummies(data['Month'])

data['winter'] = month_one_hot[[1, 2, 3, 10, 11, 12]].sum(axis=1)
data['summer'] = month_one_hot[[4, 5, 6, 7 , 9 , 10]].sum(axis=1)

data.head(1)

#### Year feature
The price increases year-to-year so the three years are one-hot encoded into three new colums (2011, 2012, and 2013)

In [ ]:
#Compute mean prices for each year
mean_price = []
for year in [2011,2012,2013]:
    mean_price.append(data['SMPEP2'][data['Year'] == year].mean())

fig, axes = plt.subplots(2, 1, figsize=(8, 5), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)
fig.subplots_adjust(hspace=0)
axes[0].plot(np.arange(0, 3), mean_price, 'k.--')
sns.boxenplot(data=data, x='Year', y='SMPEP2', ax=axes[1])
axes[0].set(ylim=(55, 75), yticks=[57.5, 62.5, 67.5, 72.5], ylabel='Average')
axes[1].set(ylim=(-25, 650));
axes[0].set_title('Price distribution across years');

#One-hot encoding of the two categories of Year
year_one_hot = pd.get_dummies(data['Year'])

if year_joined == 0:
    data = data.join(year_one_hot)
    year_joined += 1

data.head(1)

#### PeriodOfDay feature
The PeriodOfDay feature is the most important time feature as the price varies significantly during a single day. The aftenoon spikes lead to a significant increase in the mean half-hourly price between 17.00 and 18.00 hours. One-hot encoding the PeriodOfDay feature would drastically increase the dimensionality of the data set, as it would introduce 48 additional features (one for each time period). Instead, another approach is taken.

A Fourier transformation of the price feature is used to find the most significant frequency of variation. This is found to be exactly once pr. day. To emcompass this trend, the PeriodOfDay feature is encoded as two new columns: A sine and cosine with a period of one day.

In [ ]:
#Compute mean prices for periods of day
mean_price = []
for pday in list(range(0, 48)):
    mean_price.append(data['SMPEP2'][data['PeriodOfDay'] == pday].mean())

fig, axes = plt.subplots(2, 1, figsize=(12, 5), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)
fig.subplots_adjust(hspace=0)
axes[0].plot(np.arange(0, 48), mean_price, 'k.-', markersize=8)
sns.boxenplot(data=data, x='PeriodOfDay', y='SMPEP2', ax=axes[1])
axes[0].set(ylim=(30, 150), yticks=[50, 85, 120], ylabel='Average', title='Price distribution across half-hourly intervals')
axes[1].set(ylim=(-25, 650), xlabel='Hour of day');
axes[1].set(xticks=list(range(0, 48, 2)))
axes[1].set_xticklabels(list(range(0, 24)));

In [ ]:
#Find significant frequencies by Fourier transformation
fft = tf.signal.rfft(data['SMPEP2'])
frequencies = np.arange(0, len(fft))

#Change frequencies to 'pr. year'
daysinyr = 365.2524
n_samples_halfh = len(data['SMPEP2'])
halfhours_per_year = 2*24*daysinyr
years_per_dataset = n_samples_halfh/(halfhours_per_year)
frequencies_per_year = frequencies/years_per_dataset


fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.subplots_adjust(wspace=0.05)
axes = axes.ravel()
for ax in axes[0:2]:
    ax.plot(frequencies_per_year, np.abs(fft), 'k-')
    ax.set(ylim=(0, 400000), yticks=[], xticks=[], xlabel='Frequency')

axes[0].set_xscale('log')  
axes[0].set(xlim=(0.1, max(frequencies_per_year)), xticks=[1, daysinyr/12, daysinyr])
axes[0].set_xticklabels(['1/year', '1/month', '1/day']);

axes[1].set(xlim=(200, 2000), xticks=[daysinyr, daysinyr*2, daysinyr*3, daysinyr*4, daysinyr*5])
axes[1].set_xticklabels(['1/day', '2/day', '3/day', '4/day', '5/day']);

#Map the PeriodOfDay feature onto a sine and cosine function with a period of 24 hours
data['day_sin'] = np.sin(data['PeriodOfDay']*(2*np.pi/48))
data['day_cos'] = np.cos(data['PeriodOfDay']*(2*np.pi/48))

axes[2].plot(list(range(0, 48)), data['day_sin'][0:48], 'r.-', label='day sine')
axes[2].plot(list(range(0, 48)), data['day_cos'][0:48], 'g.-', label='day cosine')
axes[2].set(yticks=[], xticks=[0, 12, 24, 36, 48], xlabel='Hour of day')
axes[2].set_xticklabels([0, 6, 12, 18, 24]);
axes[2].legend();

### Machine Learning Modelling
Now that the data has been cleaned and properly encoded, it is time to train the model. We use an LSTM-based RNN as our model. For training and evaluation, we use a customary 7:2:1 training-test-validation split. Because we are using a neural network, the metadata features have to be scaled appropriately. We chose standardization over normalization since the former method of scaling is more adequate for data with significant variations, such as the afternoon spikes. It is customary that all three data sets are standardized using the mean and standard deviation of the training data set.

In [ ]:
#Define new dataframe with relevant features for modelling.
data_model = data[[
    'DateTime',
    'SMPEP2',
    'ForecastWindProduction',
    'SystemLoadEA',
    'SMPEA',
    'ORKTemperature',
    'ORKWindspeed',
    'CO2Intensity',
    'ActualWindProduction',
    'SystemLoadEP2',
    'monday',
    'midweek',
    'weekend',
    'winter',
    'summer',
     2011,
     2012,
     2013,
    'day_sin',
    'day_cos'
        ]]

In [ ]:
#Data splitting with a 7:2:1 training-test-validation split
def train_test_val_split(dataframe,
                         trainfrac=0.70,
                         testfrac=0.20,
                         valfrac=0.10):
    
    rows = len(dataframe)
    train = dataframe[0:int(rows*trainfrac)].copy().reset_index(drop=True)
    test  = dataframe[int(rows*trainfrac):int(rows*(trainfrac+testfrac))].copy().reset_index(drop=True)
    val   = dataframe[int(rows*(trainfrac+testfrac)):int(rows*(trainfrac+testfrac+valfrac))].copy().reset_index(drop=True)
    
    return train, test, val

train, test, val = train_test_val_split(data_model)
train.head(1)

In [ ]:
#Standardization of metadata features
if scaled == 0:
    train_mean = train[metadata_features].mean()
    train_std = train[metadata_features].std()
    
    train[metadata_features] = (train[metadata_features] - train_mean)/train_std
    test[metadata_features] = (test[metadata_features] - train_mean)/train_std
    val[metadata_features] = (val[metadata_features] - train_mean)/train_std

    data_scaled_feat = (data_model[metadata_features] - train_mean)/train_std
    data_scaled_feat = data_scaled_feat.melt(var_name='Feature', value_name='Stand. value')
    
    scaled += 1

fig, ax = plt.subplots(1, 1, figsize=(12, 5))
sns.violinplot(x='Feature', y='Stand. value', data=data_scaled_feat, ax=ax)
ax.set_xticklabels(data_model[metadata_features].keys(), rotation=45)
ax.set_title('Normalized metadata features', fontsize=14);

In [ ]:
#Save standardized data sets as .csv files
train.to_csv(path_or_buf=path + 'electricity_prices_train.csv')
test.to_csv(path_or_buf=path + 'electricity_prices_test.csv')
val.to_csv(path_or_buf=path + 'electricity_prices_val.csv')

#### Data batches
We are dealing with a time series forecast so, in order to make a prediction, we have to decide how many previous time steps to feed our model. We will then structure the data into data batches, where every batch will include the input features from the previous time steps plus the target value to be predicted. We shall call this the "time window".

Below, we define a class that generates the desired batches. As input, we can give the "input_width" (number of previous time steps), the "target_width" (number of time steps to be predicted), and the "target_shift", which defines how far out into the future we want to predict. We also define the feature we want to predict by "target_feature".

In [ ]:
#Class for generating batches of the time-resolved data set. Adjust input_width (# of rows in each batch),
# target_width (# of price predictions), and target_shift (# of rows between end of input and end of predictions)
class BatchGenerator():
    #Constructor
    def __init__(self, input_width=1, target_width=1, target_shift=1,
                 train_df=None, test_df=None, val_df=None,
                 target_feature=None):
        
        #Store data
        self.train_df = train_df
        self.test_df  = test_df
        self.val_df   = val_df
        
        #Feature parameters
        self.target_feature = target_feature
        self.feature_indices = {name: i for i, name in enumerate(self.train_df.columns)}
        self.target_feature_index = self.feature_indices[target_feature]
            
        #Batch parameters
        self.input_width = input_width
        self.target_width = target_width
        self.target_shift = target_shift
        
        self.total_batch_width = input_width + target_shift
        
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_batch_width)[self.input_slice]
    
        self.target_start = self.total_batch_width - self.target_width
        self.target_slice = slice(self.target_start, None)
        self.target_indices = np.arange(self.total_batch_width)[self.target_slice]
          
            
    #Method to split batches into inputs and targets
    def split_batches(self, tfdata):
        inputs = tfdata[:, self.input_slice, :]    
        targets = tf.stack([tfdata[:, self.target_slice, self.target_feature_index]], axis=-1)
        
        #Slicing does not preserve shape information so set the shapes manually
        inputs.set_shape([None, self.input_width, None])
        targets.set_shape([None, self.target_width, None])

        return inputs, targets    
    
    
    #Method to make batches from dataframe. Uses self.split_batches()
    def make_batches(self, df):
        data = np.array(df, dtype=np.float32)
        batches = tf.keras.utils.timeseries_dataset_from_array(
            data=df,
            targets=None,
            sequence_length=self.total_batch_width,
            sequence_stride=1,
            shuffle=False,
            batch_size=32
        )
        
        batches = batches.map(self.split_batches)
        return batches    
    
    
    #Method to plot batches   
    def plot(self, batch=None, plot_col=None, model=None, max_plots=3):
        
        self.batch = batch
        
        for inputs_, targets_ in self.batch:
            inputs = inputs_
            targets = targets_
        
        plot_col_index = self.feature_indices[plot_col]
        nplots = min(max_plots, len(inputs))
        
        fig, axes = plt.subplots(nplots, 1, figsize=(12, 6), sharex=True)
        fig.subplots_adjust(hspace=0)
        axes = axes.ravel()
        axes[-1].set(xlabel='Half-hours')
        
        samples = np.random.randint(0, len(targets), size=nplots)
        
        for n in range(nplots):
            axes[n].set(ylabel=plot_col, yticks=[]);
            axes[n].plot(self.input_indices, inputs[samples[n], :, plot_col_index],
                         marker='.', markersize=8, label='inputs')
            if plot_col == self.target_feature:
                axes[n].scatter(self.target_indices, targets[samples[n], :, :],
                                ec='k', label='targets', c='r', s=64)
        
            if model is not None:
                predictions = model(inputs)
                axes[n].scatter(self.target_indices, predictions[samples[n], :],
                            marker='X', ec='k', label='predictions', c='#ff7f0e', s=64)
            if n == 0:
                axes[n].legend()
    
    
    #Object properties to access and make batches of training, test or validation
    #uses self.make_batches()
    @property
    def train(self):
        return self.make_batches(self.train_df)
    
    @property
    def test(self):
        return self.make_batches(self.test_df)
    
    @property
    def val(self):
        return self.make_batches(self.val_df)
    
    #Represent method
    def __repr__(self):
        return '\n'.join([
            f'Total batch width  {self.total_batch_width}',
            f'Input indices      {self.input_indices}',
            f'Target indices     {self.target_indices}',
            f'Target feature     {self.target_feature}'])

We can now use the BatchGenerator() class to generate batches with a desired time window. Here, we use a window with 24 time steps (corresponding to 12 hours) to predict a single target value one step into the future for the 'SMPEP2' feature. 

We want to be able to experiment with different time windows and model architectures, so we define a function to compile, fit, and save a particular model operated on a particular structure of data.

In [ ]:
#Initialize batches as objects of the BatchGenerator class
window24 = BatchGenerator(input_width=24, target_width=1, target_shift=1, target_feature='SMPEP2',
                           train_df=train.select_dtypes(exclude='datetime64[ns]'),
                           test_df=test.select_dtypes(exclude='datetime64[ns]'),
                           val_df=val.select_dtypes(exclude='datetime64[ns]')
                         )

window24

In [ ]:
#Function to compile, fit, and, save a specified model
def compile_and_fit(model=None, data_batches=None, num_epochs=10, model_nm='my_model'):
    
    if model is not None:
        model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])
    
    if data_batches is not None:
        history = model.fit(data_batches.train, epochs=num_epochs,
                          validation_data=data_batches.val)
    
    if not os.path.exists(path+'\models/'):
        os.mkdir(path+'\models/')
    
    model.save(path + '\models/' + model_nm)
    print('Model saved in:', path + '\models/' + model_nm)
        
    return history

#### Machine Learning Model: LSTM-based RNN
Now, we are ready to build the LSTM-based RNN. We choose a 4-LSTM-layered RNN where each layer consists of 100 neurons with a dropout rate of 0.2. We shall train it for 100 epochs. After training, we evaluate the performance on the training, test, and validation data set, and save the results to a text file, alongside the model architecture.

In [ ]:
#Build Recurrent Neural Network (RNN)
datasetup = window24
model_name = '4layers_100each_drop02_window24'
num_epochs = 100

RNN = Sequential()

#1st layer
RNN.add(layers.LSTM(units=100, return_sequences=True,
                            input_shape=(datasetup.input_width, len(datasetup.feature_indices))))
RNN.add(layers.Dropout(0.2))

#2nd layer
RNN.add(layers.LSTM(units=100, return_sequences=True))
RNN.add(layers.Dropout(0.2))

#3rd layer
RNN.add(layers.LSTM(units=100, return_sequences=True))
RNN.add(layers.Dropout(0.2))

#4th layer
RNN.add(layers.LSTM(units=100, return_sequences=False))
RNN.add(layers.Dropout(0.2))

#output layer
RNN.add(layers.Dense(units=1))

In [ ]:
#Compile, fit, and save a model with specified dataset
history = compile_and_fit(model=RNN,
                          data_batches=datasetup,
                          num_epochs=num_epochs,
                          model_nm=model_name)

In [ ]:
#Save model summary and performance as .txt file
model = RNN

performance_train = model.evaluate(datasetup.train)
performance_test  = model.evaluate(datasetup.test)
performance_val   = model.evaluate(datasetup.val)

with open(path + '\models/' + model_name + '.txt', "a") as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write(f"\nEpochs = {num_epochs}")
    f.write('\n\nData\tLoss\tMAE\n')
    f.write(f"train\t{performance_train[0]:.4f}\t{performance_train[1]:.4f}\n")
    f.write(f"test\t{performance_test[0]:.4f}\t{performance_test[1]:.4f}\n")
    f.write(f"val\t{performance_val[0]:.4f}\t{performance_val[1]:.4f}\n")


### Model performance and evaluation
We have evaluated the model in terms of the mean absolute error (MAE) on the training, test, and validation data sets. To get a visual understanding of the performance, we can use the plot() method from the BatchGenerator() class. Below, three random data batches from the test set have been plotted. 

However, we are mostly interested in the performance on an entire day, and whether or not the model can forecast a price spike. 

We make a new dataframe where we recombine the date and time from the DateTime feature with the original forecast (SMPEA), the target prices (SMPEP2), and the model predictions. The prices were scaled for training of the model, so we have to remember to rescale to the original values. Now, we can plot the performance on selected dates.

At first sight, the model predictions seem to be more accurate than the original forecast. We have check this for all dates to be sure, however. Therefore, we compute the daily MAE both for the predictions and the original forecast for every date in the test data set. We can also compute the mean of the daily MAE, i.e. the total MAE. This turns out to be 13.93 for the original forecast versus 6.64 for the model predictions, which tells us that we have more than halfed the error on the original forecast.

In [ ]:
#Plot inputs, tagets and predictions for random data batches in the test data set
datasetup.plot(batch=datasetup.test.take(1), plot_col='SMPEP2', model=model)

In [ ]:
#Make new dataframe with orignal forecast, target prices, and model predictions for the test data set
predicts = model.predict(datasetup.test)

forecast_df = pd.DataFrame()
forecast_df.insert(loc=0, column='Date', value=test['DateTime'].apply(lambda x: x.date()))
forecast_df.insert(loc=1, column='Time', value=test['DateTime'].apply(lambda x: x.time()))

forecast_df.loc[:,'Original'] = test['SMPEA'].apply(lambda x: x*train_std['SMPEA'] + train_mean['SMPEA'])
forecast_df.loc[datasetup.input_width:,'Target'] = test['SMPEP2'].apply(lambda x: x*train_std['SMPEP2'] + train_mean['SMPEP2'])
forecast_df.loc[datasetup.input_width:,'Prediction'] = predicts*train_std['SMPEP2'] + train_mean['SMPEP2']

# Plot selected dates with original forecasts, target prices, and model predictions
dates = forecast_df['Date'].unique()[1::]
plot_dates = [dates[11], dates[112]]


x = list(range(0, 48))
for date in plot_dates:
    fig, ax = plt.subplots(1, 1, figsize=(12,2.5))
    ax.plot(x, forecast_df['Original'][forecast_df['Date'] == date], c='grey', marker='x', label='original')
    ax.plot(x, forecast_df['Target'][forecast_df['Date'] == date], c='r', marker='o', label='targets')
    ax.plot(x, forecast_df['Prediction'][forecast_df['Date'] == date], c='g', marker='s', label='predictions')
    ax.set(ylabel='Price', xlabel='Time of Day', xticks=list(range(0, 49, 2)))
    ax.set_xticklabels(list(range(0, 25)))
    ax.set_title(str(days_name[date.weekday()]) + ' ' + str(date), fontsize=14)
    ax.legend();

In [ ]:
#Compute and plot the MAE for the original forecast and the model predictions
originalDailyMAE = []
predictionsDailyMAE = []
for date in dates:      
    originals = np.array(forecast_df['Original'][forecast_df['Date'] == date])
    targets = np.array(forecast_df['Target'][forecast_df['Date'] == date])
    predictions = np.array(forecast_df['Prediction'][forecast_df['Date'] == date])
    
    originalDailyMAE.append(np.sum(np.abs(targets - originals))/len(targets))
    predictionsDailyMAE.append(np.sum(np.abs(targets - predictions))/len(targets))

fig, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(list(range(len(dates))), originalDailyMAE, c='r', alpha=0.3, marker='x', label='original MAE')
ax.plot(list(range(len(dates))), predictionsDailyMAE, c='g', marker='x', label='prediction MAE');

ax.set(ylabel='Daily MAE of price', xlabel='Date in test set');
ax.set(ylim = (0, 35))
ax.legend()

#Find the total MAE for all dates
originalMAE = np.sum(originalDailyMAE)/len(dates)
predictionsMAE = np.sum(predictionsDailyMAE)/len(dates)

print('Original MAE          {:.2f} ({:.2f}% of mean target price)'.format(originalMAE, originalMAE/mean_actual_price*100))
print('Model predictions MAE {:.2f}  ({:.2f}% of mean target price)'.format(predictionsMAE, predictionsMAE/mean_actual_price*100))